In [15]:
import numpy as np
from src.darwin_core_aggregations import DarwinCoreAggregations
from src.geojson import build_geojson_feature_collection
import numpy as np
from contexttimer import Timer
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist
from src.cluster_color_builder import ClusterColorBuilder
from src.cluster_stats import Stats
from src.darwin_core_aggregations import DarwinCoreAggregations
from src.render import plot_clusters
from src.cluster import ClusterId
import matplotlib.pyplot as plt
from src.geojson import build_geojson_feature_collection
import os
import logging
from sklearn.preprocessing import StandardScaler
from run import cluster, build_X


In [16]:
# Inputs
input_file = "data-new-york-city.txt"
geohash_precision = 4
show_dendrogram = True
log_file = "log.txt"
output_file = "output.geojson"
num_clusters = 2
use_cache = False
plot = True

In [17]:
# set up logging
logging.basicConfig(filename=log_file, encoding="utf-8", level=logging.INFO)


In [18]:
darwin_core_aggregations = DarwinCoreAggregations.build(
    input_file, geohash_precision
)

In [19]:
darwin_core_aggregations.unfiltered_taxon_counts

geohash,kingdom,rank,name,count
str,enum,enum,str,u32
"""dr5""","""Plantae""","""species""","""Wisteria sinensis""",21
"""dr5""","""Animalia""","""genus""","""Xenox""",229
"""dr5""","""Animalia""","""genus""","""Stenotomus""",14
"""dr5""","""Animalia""","""species""","""Dilophus spinipes""",113
"""dr5""","""Animalia""","""genus""","""Rhaphium""",1
…,…,…,…,…
"""dr7""","""Animalia""","""species""","""Nadata gibbosa""",9
"""dr7""","""Fungi""","""species""","""Gymnopus junquilleus""",1
"""dr7""","""Plantae""","""order""","""Cornales""",265


In [20]:
darwin_core_aggregations.taxon_counts

geohash,kingdom,species,count
str,enum,str,u32
"""dr5""","""Plantae""","""Wisteria sinensis""",21
"""dr5""","""Animalia""","""Dilophus spinipes""",113
"""dr5""","""Animalia""","""Oecanthus nigricornis""",6
"""dr5""","""Plantae""","""Linum usitatissimum""",1
"""dr5""","""Animalia""","""Renia factiosalis""",1
…,…,…,…
"""dr7""","""Fungi""","""Cortinarius albobrunnescens""",1
"""dr7""","""Animalia""","""Nadata gibbosa""",9
"""dr7""","""Fungi""","""Gymnopus junquilleus""",1


In [21]:

cluster_dataframe = cluster(
    darwin_core_aggregations,
    num_clusters,
    show_dendrogram,
    use_cache,
)

/Users/coreyf/dev/frewsxcv/citizen-science-ecoregions-experiments/.venv/lib/python3.13/site-packages/sklearn/decomposition/_incremental_pca.py:376: RuntimeWarning: Mean of empty slice.
  self.noise_variance_ = explained_variance[self.n_components_ :].mean()
/Users/coreyf/dev/frewsxcv/citizen-science-ecoregions-experiments/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: The condensed distance matrix must contain only finite values.

In [ ]:


# Find the top averages of taxon
all_stats = Stats.build(darwin_core_aggregations)

feature_collection = build_geojson_feature_collection(
    (
        cluster,
        geohashes,
        cluster_dataframe.determine_color_for_cluster(
            cluster, darwin_core_aggregations
        ),
    )
    for cluster, geohashes in cluster_dataframe.iter_clusters_and_geohashes()
)

# print_results(darwin_core_aggregations, all_stats, cluster_dataframe)

plot_clusters(feature_collection, num_clusters=num_clusters)

# write_geojson(feature_collection, args.output_file)

In [ ]:
from run import print_cluster_stats
cluster_id = 1
geohashes = cluster_dataframe.geohashes_for_cluster(cluster_id)
print_cluster_stats(cluster_id, geohashes, darwin_core_aggregations, all_stats)


In [ ]:

stats_for_clusters = {
    cluster: Stats.build(
        darwin_core_aggregations,
        geohash_filter=geohashes,
    )
    for cluster, geohashes in cluster_dataframe.iter_clusters_and_geohashes()
}
stats_for_clusters


In [ ]:
[
    {
        "cluster": cluster,
        "waterfowl": stats.waterfowl_count(),
        "aves": stats.aves_count(),
        "ratio": stats.waterfowl_count() / stats.aves_count(),
    }
    for cluster, stats in stats_for_clusters.items()
]

